In [10]:
!pip install monai

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 8.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 29.7 MB/s eta 0:00:0000:0100:01
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
  Attempting uninstall: torch
    Found existing installation: torch 2.3.0
    Uninstalling torch-2.3.0:
      Successfully uninstalled torch-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.18.1 requires torch==2.3.1, but you have torch 2.9.1 which is incompatible.


In [11]:
from snowflake.snowpark import Session
from snowflake.snowpark.context import get_active_session
from dotenv import load_dotenv

import os
load_dotenv() # This loads the .env file

from monai.apps import download_url, download_and_extract


# Get active session (automatically available in Snowflake Notebooks)
connection_parameters = dict(
    account = 'sfsenorthamerica-cguzman_aws_us_west_2',
    user = 'carlos.guzman@snowflake.com',
    password = os.getenv('SNOWFLAKE_PASSWORD'),
    role = 'sysadmin',
    warehouse = 'compute_wh',
    database = 'sf_clinical_db',
    schema = 'utils'
 )
 
try:
    session = get_active_session()
except:
    session = Session.builder.configs(connection_parameters).create()




/Users/cguzman/miniconda3/envs/py_spcs/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1120 15:10:33.133000 79784 site-packages/torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [12]:
session.sql("select current_user()").collect()

[Row(CURRENT_USER()='CGUZMAN')]

In [13]:
import tempfile

directory = '/tmp'
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

/tmp


In [14]:
resource = "https://zenodo.org/record/3835682/files/training.zip"

compressed_file = os.path.join(root_dir, "paired_ct_lung.zip")
data_dir = os.path.join(root_dir, "paired_ct_lung")
if not os.path.exists(data_dir):
    download_and_extract(resource, compressed_file, root_dir)
    os.rename(os.path.join(root_dir, "training"), data_dir)

paired_ct_lung.zip: 266MB [00:38, 7.18MB/s]                               

2025-11-20 15:11:28,678 - INFO - Downloaded: /tmp/paired_ct_lung.zip
2025-11-20 15:11:28,679 - INFO - Expected md5 is None, skip md5 check for file /tmp/paired_ct_lung.zip.
2025-11-20 15:11:28,679 - INFO - Writing into directory: /tmp.


In [19]:
session.use_database("SF_CLINICAL_DB")
session.use_schema("UTILS")


session.file.put(f"{data_dir}/lungMasks/*exp.nii.gz", "@monai_medical_images_stg/lungMasksExp", overwrite=True)
session.file.put(f"{data_dir}/lungMasks/*insp.nii.gz", "@monai_medical_images_stg/lungMasksInsp", overwrite=True)
session.file.put(f"{data_dir}/scans/*exp.nii.gz", "@monai_medical_images_stg/scansExp", overwrite=True)
session.file.put(f"{data_dir}/scans/*insp.nii.gz", "@monai_medical_images_stg/scansInsp", overwrite=True)



[PutResult(source='case_001_insp.nii.gz', target='case_001_insp.nii.gz', source_size=8197409, target_size=8197409, source_compression='GZIP', target_compression='GZIP', status='UPLOADED', message=''),
 PutResult(source='case_002_insp.nii.gz', target='case_002_insp.nii.gz', source_size=9557874, target_size=9557874, source_compression='GZIP', target_compression='GZIP', status='UPLOADED', message=''),
 PutResult(source='case_003_insp.nii.gz', target='case_003_insp.nii.gz', source_size=9206735, target_size=9206735, source_compression='GZIP', target_compression='GZIP', status='UPLOADED', message=''),
 PutResult(source='case_004_insp.nii.gz', target='case_004_insp.nii.gz', source_size=7812881, target_size=7812881, source_compression='GZIP', target_compression='GZIP', status='UPLOADED', message=''),
 PutResult(source='case_005_insp.nii.gz', target='case_005_insp.nii.gz', source_size=9245125, target_size=9245125, source_compression='GZIP', target_compression='GZIP', status='UPLOADED', message=